### Create dsQ job submission array
#### Commands: 


ml load dSQ/1.05

1: dsq --job-file 1_preproc_dsq_job_array.txt --mem-per-cpu 4g -t 1:00:00 --mail-type ALL --partition gpu --cpus-per-task=1 --gres=gpu:2

3: dsq --job-file 2_csd_dsq_job_array.txt --mem-per-cpu 10g -t 10:00:00 --mail-type ALL --partition verylong


4: dsq --job-file 4_tract_dsq_job_array.txt --mem-per-cpu 10g -t 10:00:00 --mail-type ALL --partition verylong


In [1]:
import pandas as pd
import csv
from glob import glob

In [2]:
home = '/home/lms233/Github/Diffusion'
candpath = '/gpfs/milgram/pi/gee_dylan/candlab'
data = candpath + '/analyses/shapes/dwi/data/'

In [3]:
subject_info = pd.Series(glob(data + '/3_Eddy_Corrected/sub*'))
subs = subject_info.str.replace(data + '/3_Eddy_Corrected/', '')
all_subjects = subs.tolist()

In [5]:
#Generate subs who didn't process fully
subjslist = []
for i in range(0, len(all_subjects)):
    sub = all_subjects[i]
    newlist = pd.Series(glob(data+ '/4_Deconvolution/*'))
    newsubs = newlist.replace(data + '/4_Deconvolution/', '').tolist()
    if sub in newsubs:
        pass
    else:
        subjslist.append(sub)

notrun_subjects = list(set(subjslist))

/gpfs/milgram/project/gee_dylan/lms233/conda_envs/shapes-dwi3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [6]:
print(len(subs))
print(len(notrun_subjects))

175
175


### Create batch file for preprocessing

In [9]:
#Set subjects
subjects = all_subjects
commands_preproc = []

for i in range(0, len(subjects)):
    sub = subjects[i]
    commands_preproc.append('sh 1_sbatch_preproc.sh {}'.format(sub))

In [10]:
out = pd.DataFrame(commands_preproc)
out.to_csv(home + '/1_preproc_dsq_job_array.txt', sep = '\t', header = False, index=False, 
           quoting=csv.QUOTE_NONE)

### Create batch file for MS CSD

In [7]:
#Set subjects
subjects = all_subjects

commands_csd = []

for i in range(0, len(subjects)):
    sub = subjects[i]
    commands_csd.append('sh 2_sbatch_csd.sh {}'.format(sub))

In [8]:
out = pd.DataFrame(commands_csd)
out.to_csv(home + '/2_csd_dsq_job_array.txt', sep = '\t', header = False, index=False, 
           quoting=csv.QUOTE_NONE)

### Create batch file for TractSeg

In [34]:
subjects = all_subjects
commands_tract = []

for i in range(0, len(subjects)):
    sub = subjects[i]
    commands_tract.append('sh 4_TractSegFlow.sh {}'.format(sub))

In [35]:
out_tract = pd.DataFrame(commands_tract)
out_tract.to_csv(home + '/4_tract_dsq_job_array.txt', sep = '\t', header = False, index=False, 
           quoting=csv.QUOTE_NONE)